In [1]:
import requests
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
import os
from datetime import datetime as dt
import os.path
from os import path
import sys
from importlib import reload
import copy
import pickle

In [2]:
sys.path.append('../..') #agregar directorio madre del repo para poder importar librerias customizadas
#importar librerias customizadas

In [3]:
import modules.Elastic_Module
reload(modules.Elastic_Module)

<module 'modules.Elastic_Module' from '../../modules/Elastic_Module.py'>

In [4]:
from modules.Elastic_Module import *

In [5]:
url_ckan='http://datos.gob.cl/api/3/action/package_list'
url_package='http://datos.gob.cl/api/3/action/package_show?id='
url_package_search='http://datos.gob.cl/api/3/action/package_search?'

In [6]:
#Listas de paquetes ckan (con sus ID)
ckan_packages=requests.get(url_ckan) #consultar api
ckan_packages=ckan_packages.json() #convertir respuesta a json

In [57]:
ckan_packages['result'][0:15]

['10000',
 '10002',
 '10006',
 '10007',
 '10010',
 '10011',
 '10014',
 '10029',
 '10037',
 '10043',
 '10049',
 '10054',
 '10063',
 '10069',
 '10071']

In [8]:
#OBTENER TODOS LOS PAQUETES DE LA BASE DE DATOS CKAN
rows=str(1000) #maximo numero de resultados por consulta (maximo permitido es 1000)
packs=[] #contenedor de paquetes retornados
for s in range(0,4):
    start=str(s*int(rows)) #paginacion
    packages=requests.get(url_package_search+'rows='+rows+'&'+'start='+start)
    packages=packages.json()
    packs.extend(packages['result']['results'])

In [9]:
#revisar root de los paquetes
#root={}
#for k,v in packs[0].items():
#    if (k!='resources') & (k!='organization') :
#        root[k]=v

In [10]:
recursos=[]
for p in packs:
    for r in p['resources']:
        d=copy.deepcopy(r)
        #parámetros del paquete
        d['package_id']=p['id'] #id
        d['package_name']=p['name'] #nombre
        d['package_title']=p['title'] #titulo
        d['package_notes']=p['notes'] #notas
        if 'organization' in p.keys(): #si es que existe info de organizacion
            if p['organization']!=None:
                d['inst.']=p['organization']['title'] #si es que existe info del titulo incluirlo
                d['inst_name']=p['organization']['name'] #si es que existe info del titulo incluirlo
                d['inst_id']=p['organization']['id'] #si es que existe info del id inst.
            else:
                d['inst.']=None
                d['inst_name']=None
                d['inst_id']=None
        recursos.append(d)
recursos=pd.DataFrame(recursos) #convertir a df        

In [11]:
recursos=recursos.rename(columns={'last_modified':'fecha','format':'formato'}) #cambiar nombres de columnas

In [12]:
recursos=recursos.drop(['mimetype','cache_url','url','cache_last_updated','hash','url_type','webstore_url','webstore_last_updated',
              'mimetype_inner','position','resource_type'],axis=1) #botar columnas

In [13]:
recursos['fecha']=pd.to_datetime(recursos['fecha']) #convertir la fecha a formato solo con la fecha (sin la hora)
#desagregar en dia, mes, año
recursos['dia']=recursos['fecha'].dt.date
recursos['year']=recursos['fecha'].dt.year
recursos['mes']=recursos['fecha'].dt.month

In [14]:
#guardar paso en csv
path='/data/cache/ckan/state_datos_gob'
now = dt.now()
date_time = now.strftime("_%m-%d-%Y_%H-%M-%S")

In [15]:
recursos.to_csv(path+date_time+'.csv')

In [16]:
#guardar data de paquetes cruda en pickle y borrar para liberar espacio en memoria
with open(path+date_time+'.pickle', 'wb') as f:
    pickle.dump(packs, f)
del(packs)

In [17]:
#RUTA PARA RECUPERAR PASO GUARDADO EN CSV
#ls /data/cache/ckan/state_datos_gob*

In [18]:
#RECUPERAR PASO GUARDADO EN CSV
#recursos=pd.read_csv('/data/cache/ckan/state_datos_gob_03-31-2020_05-05-15.csv',index_col=0)

In [19]:
data=recursos.to_dict(orient='records')

In [20]:
#variables de entorno
args={}
args['elastic_user']='elastic'
args['elastic_psswd']=os.environ['ELASTIC_PASSWD']
args['DOMAIN_ELASTIC']=os.environ['DOMAIN_ELASTIC']
args['ELASTIC_PORT']=os.environ['ELASTIC_PORT']

In [26]:
ES_HOST = "https://"+args['DOMAIN_ELASTIC']
basic_auth={'user':args['elastic_user'], 'psswd':args['elastic_psswd']}

In [35]:
es=elastic_connection(ES_HOST,basic_auth=basic_auth)

In [36]:
es.ping()

True


In [37]:
#generar campos concatenados (parámetros)
generateID_params_custom={'fields':['package_id','id'], #campos a concatenar
                               'head':20, #numero de caracteres a dejar
                                'sep':'_' #separador entre campos
                               }

In [38]:
es.generateID_params_custom=generateID_params_custom

In [39]:
es.add_docs(recursos)

In [44]:
indexes=[
    ('ckan','ckan')
        ]

In [45]:
#generar bulk data
i=indexes[0]
n=1500
id_field='id'
es.bulk_data(INDEX_NAME=i[0],_type=i[1],n=n,id_field=id_field)
#test

creando paquetes de data bulk
size of bulk docs is : 0.1953125 Kbs
N° docs: 10634.0, N° chunks: 15


In [46]:
len(es.chunks[0])

1500

In [47]:
#prepar la data para elastic search
es.elastic_prepare_pattern()

In [48]:
#es.chunks[0]

In [53]:
es.clear_index(i[0])

deleting 'ckan' index...
 response: '{'acknowledged': True}'
creating 'ckan' index...
 response: '{'acknowledged': True, 'shards_acknowledged': True, 'index': 'ckan'}'


In [54]:
es.bulk_index()

bulk indexing...:index---> ckan
 indexing chunk : 0 of 15

../../modules/Elastic_Module.py:140: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  res = es.bulk(index = INDEX_NAME_, body = chunks[idx],refresh = True)


 indexing chunk : 14 of 15finished bulk indexing
No recorded ES indexing errors
No recorded ES indexing exceptions


In [55]:
es.test_index(i[0],n=15)

../../modules/Elastic_Module.py:78: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize([x['_source'] for x in res['hits']['hits']])


,description,name,formato,package_id,created,state,fecha,revision_id,size,package_name,package_title,package_notes,inst.,inst_name,inst_id,dia,year,mes
0,Archivo de registros de ítems correspondientes...,Exportaciones Enero 2020,RAR,8ffb8730-8408-4db1-8217-c0fa2a955cff,2020-02-27T11:40:30.836089,active,2020-02-27T14:40:30.733565,ca4e37f5-6b6c-4b73-887f-cf8f6f175160,None,registro-de-exportacion-2020,Registro de Exportación 2020,Muestra el registro de ítems correspondientes ...,Servicio Nacional de Aduanas,servicio_nacional_de_aduanas,AE007,2020-02-27,2020.0,2.0
1,Archivo de registros de ítems correspondientes...,Exportaciones Enero 2020 – Bultos,RAR,8ffb8730-8408-4db1-8217-c0fa2a955cff,2020-02-27T11:44:51.362792,active,2020-02-27T14:44:51.262023,ca4e37f5-6b6c-4b73-887f-cf8f6f175160,None,registro-de-exportacion-2020,Registro de Exportación 2020,Muestra el registro de ítems correspondientes ...,Servicio Nacional de Aduanas,servicio_nacional_de_aduanas,AE007,2020-02-27,2020.0,2.0
2,Archivo de registros de ítems correspondientes...,Exportaciones Enero 2020 – Documentos de Trans...,RAR,8ffb8730-8408-4db1-8217-c0fa2a955cff,2020-02-27T11:45:52.687192,active,2020-02-27T14:45:52.586540,ca4e37f5-6b6c-4b73-887f-cf8f6f175160,None,registro-de-exportacion-2020,Registro de Exportación 2020,Muestra el registro de ítems correspondientes ...,Servicio Nacional de Aduanas,servicio_nacional_de_aduanas,AE007,2020-02-27,2020.0,2.0
3,Archivo de registros de ítems correspondientes...,Exportaciones Febrero 2020,RAR,8ffb8730-8408-4db1-8217-c0fa2a955cff,2020-04-01T12:53:37.909996,active,2020-04-01T15:53:37.801486,843bfea9-ad50-4701-bce6-0b6acea9c080,None,registro-de-exportacion-2020,Registro de Exportación 2020,Muestra el registro de ítems correspondientes ...,Servicio Nacional de Aduanas,servicio_nacional_de_aduanas,AE007,2020-04-01,2020.0,4.0
4,Archivo de registros de ítems correspondientes...,Exportaciones Febrero 2020 - Bultos,RAR,8ffb8730-8408-4db1-8217-c0fa2a955cff,2020-04-01T12:54:55.264410,active,2020-04-01T15:56:10.799996,10792899-d3ed-49c3-9a67-3eb2a3dc61fb,None,registro-de-exportacion-2020,Registro de Exportación 2020,Muestra el registro de ítems correspondientes ...,Servicio Nacional de Aduanas,servicio_nacional_de_aduanas,AE007,2020-04-01,2020.0,4.0
5,Archivo de registros de ítems correspondientes...,Exportaciones Febrero 2020 - Documentos de Tra...,RAR,8ffb8730-8408-4db1-8217-c0fa2a955cff,2020-04-01T12:57:48.850237,active,2020-04-01T15:57:48.751851,40cf1794-9c94-4be2-a082-34d4fe453762,None,registro-de-exportacion-2020,Registro de Exportación 2020,Muestra el registro de ítems correspondientes ...,Servicio Nacional de Aduanas,servicio_nacional_de_aduanas,AE007,2020-04-01,2020.0,4.0
6,Archivo de registros de ítems correspondientes...,Exportaciones Marzo 2020,RAR,8ffb8730-8408-4db1-8217-c0fa2a955cff,2020-04-29T18:30:59.234093,active,2020-04-29T21:30:59.143689,603371a4-e55c-4953-a779-1c91169bfcd3,None,registro-de-exportacion-2020,Registro de Exportación 2020,Muestra el registro de ítems correspondientes ...,Servicio Nacional de Aduanas,servicio_nacional_de_aduanas,AE007,2020-04-29,2020.0,4.0
7,Archivo de registros de ítems correspondientes...,Exportaciones Marzo 2020 - Bultos,RAR,8ffb8730-8408-4db1-8217-c0fa2a955cff,2020-04-29T18:31:51.519550,active,2020-04-29T21:31:51.422127,d0cc62ec-1786-4bfe-9111-7d572029193d,None,registro-de-exportacion-2020,Registro de Exportación 2020,Muestra el registro de ítems correspondientes ...,Servicio Nacional de Aduanas,servicio_nacional_de_aduanas,AE007,2020-04-29,2020.0,4.0
8,Archivo de registros de ítems correspondientes...,Exportaciones Marzo 2020 - Documentos de Trans...,RAR,8ffb8730-8408-4db1-8217-c0fa2a955cff,2020-04-29T18:32:59.316328,active,2020-04-29T21:32:59.195983,cdb89728-af32-4225-92cb-8022ca303f45,None,registro-de-exportacion-2020,Registro de Exportación 2020,Muestra el registro de ítems correspondientes ...,Servicio Nacional de Aduanas,servicio_nacional_de_aduanas,AE007,2020-04-29,2020.0,4.0
9,Archi

In [56]:
es.count_documents(i[0])

[{'epoch': '1593568120', 'timestamp': '01:48:40', 'count': '10558'}]
